In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached protobuf-4.24.1-cp37-abi3-manylinux2014_x86_64.whl (311 kB)
  Using cached tensorflow_io_gcs_filesystem-0.33.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached grpcio-1.57.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached wrapt-1.15.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (78 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (1

In [3]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached matplotlib-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 23.5 MB/s eta 0:00:0000:0100:01
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from tensorflow import keras
from tensorflow.keras import layers
file1 = open('/home/ikm/Downloads/IMDB Dataset.csv', 'r').readlines()
fileout = open('/home/ikm/Downloads/IMDB Dataset1.csv', 'w')
for line in file1:
    fileout.write(line.replace('<br />',' '))#remove <br />

fileout.close()
dataname = "/home/ikm/Downloads/IMDB Dataset1.csv"

data = pd.read_csv(dataname)
data['sentiment']=data['sentiment']
data['sentiment']=data['sentiment'].map({'positive':1,'negative':0})
print(data.head())

X=data['review'].values
Y=data['sentiment'].values

2023-08-24 10:12:29.886037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 10:12:30.437847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production.   The filming t...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1


In [2]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
import tensorflow as tf
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data) #轉小寫
    return tf.strings.regex_replace(
        lowercase, "[%s]" % re.escape(string.punctuation), "" #去除標點符號
    )


# 參數設定
max_features = 99421 #考慮字彙數量
embedding_dim = 256  #每個查詢向量維度
sequence_length = 500 #輸入資料長度


vectorize_layer = TextVectorization(
    standardize=custom_standardization, #轉小寫資料
    max_tokens=max_features,
    output_mode="int", #輸出為int
    output_sequence_length=sequence_length,
)
vectorize_layer.adapt(X)

2023-08-24 10:12:33.118623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 10:12:33.118928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 10:12:33.150088: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the 

In [3]:
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2)

In [4]:
print(X_train.shape)
print(X_test.shape)


(40000,)
(10000,)


In [5]:
type(X_train)

numpy.ndarray

In [9]:
from sklearn.metrics import f1_score

# 自定义F1 Score指标
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        true_pos = tf.reduce_sum(y_true * y_pred)
        false_pos = tf.reduce_sum((1 - y_true) * y_pred)
        false_neg = tf.reduce_sum(y_true * (1 - y_pred))
        self.true_positives.assign_add(true_pos)
        self.false_positives.assign_add(false_pos)
        self.false_negatives.assign_add(false_neg)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
        return f1
from tensorflow.keras import layers

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)  #將資料向量化
x = layers.Embedding(max_features + 1, embedding_dim)(x) #維度128
x = layers.Dropout(0.5)(x)
x = layers.Conv1D(256, 7, padding="valid", activation="relu", strides=3)(x) #一維捲積層 strides=移動步長
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x) #池化層

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x) #分類

model = tf.keras.Model(text_input, predictions)

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])
print(model.summary())
for layer in model.layers:
    print(layer.name, layer.input_shape)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text (InputLayer)           [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 500)               0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 500, 256)          25452032  
                                                                 
 dropout_2 (Dropout)         (None, 500, 256)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 165, 256)          459008    
                                                                 
 conv1d_4 (Conv1D)           (None, 53, 128)           229504    
                                                           

In [12]:
epochs = 3
his = model.fit(X_train,Y_train,batch_size = 128,validation_split = 0.2, epochs=epochs)


Epoch 1/3


250/250 [==============================] - 116s 465ms/step - loss: 0.0240 - accuracy: 0.9923 - precision: 0.9920 - recall: 0.9926 - f1_score: 0.9873 - val_loss: 0.4848 - val_accuracy: 0.8848 - val_precision: 0.8866 - val_recall: 0.8835 - val_f1_score: 0.8777
Epoch 2/3
250/250 [==============================] - 116s 464ms/step - loss: 0.0152 - accuracy: 0.9952 - precision: 0.9953 - recall: 0.9951 - f1_score: 0.9922 - val_loss: 0.4772 - val_accuracy: 0.8825 - val_precision: 0.8695 - val_recall: 0.9011 - val_f1_score: 0.8771
Epoch 3/3
250/250 [==============================] - 116s 464ms/step - loss: 0.0106 - accuracy: 0.9965 - precision: 0.9966 - recall: 0.9963 - f1_score: 0.9942 - val_loss: 0.6253 - val_accuracy: 0.8801 - val_precision: 0.8569 - val_recall: 0.9138 - val_f1_score: 0.8807


In [11]:
print(model.evaluate(X_test,Y_test))

313/313 [==============================] - 4s 14ms/step - loss: 0.3457 - accuracy: 0.8906 - precision: 0.8767 - recall: 0.9064 - f1_score: 0.8786
[0.34569454193115234, 0.8906000256538391, 0.8766855597496033, 0.9064457416534424, 0.8785926699638367]


In [9]:
print(X_test.shape)

(10000,)
